In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import numpy as np
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import roc_auc_score, confusion_matrix, classification_report
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as imPipeline
from xgboost import XGBClassifier
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV
from sklearn.metrics import roc_curve, auc, confusion_matrix, precision_recall_fscore_support, balanced_accuracy_score
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as imPipeline

In [ ]:
def evaluate_model(model, X_test, y_test):
    predictions = model.predict(X_test)
    print("Unique labels in y_test:", np.unique(y_test))
    print("Unique labels in predictions:", np.unique(predictions))

    probs = model.predict_proba(X_test)[:, 1]

    fpr, tpr, thresholds = roc_curve(y_test, probs)
    roc_auc = auc(fpr, tpr)
    cm = confusion_matrix(y_test, predictions)

    if cm.shape == (1, 1):
        # Only one class present in y_test and/or predictions
        tn, fp, fn, tp = (cm[0, 0], 0, 0, 0) if np.unique(y_test)[0] == 0 else (0, 0, 0, cm[0, 0])
    else:
        tn, fp, fn, tp = cm.ravel()
    fpr = fp / (fp + tn)
    fnr = fn / (fn + tp)
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    youdens_index = sensitivity + specificity - 1
    precision, recall, f1_score, _ = precision_recall_fscore_support(y_test, predictions, average='macro')
    balanced_accuracy = balanced_accuracy_score(y_test, predictions)

    results = {
        'FPR': fpr,
        'FNR': fnr,
        'AUC': roc_auc,
        'F1 Score': f1_score,
        'Recall': recall,
        'Precision': precision,
        'Balanced Accuracy': balanced_accuracy,
        'Youden\'s Index': youdens_index
    }
    return results

In [ ]:
# Load the dataset
datasets=['imdb','sst2','ag-news','yelp_polarity']


for dataset_name in datasets:
  models=[]
  if(dataset_name in ['imdb','ag-news']):

    models.append('textattack/bert-base-uncased-'+dataset_name)
    models.append('textattack/roberta-base-'+dataset_name)
  elif(dataset_name =='sst2'):
    models.append('textattack/bert-base-uncased-SST-2')
    models.append('textattack/roberta-base-SST-2')

  else:
    models.append('textattack/bert-base-uncased-yelp-polarity')
    models.append('VictorSanh/roberta-base-finetuned-yelp-polarity')

  print(dataset_name)
  print(models)
  for model_name in models:
        path = '/content/gdrive/My Drive/GLAD_RESULTS/' + dataset_name + '/'
        savepath='/content/gdrive/My Drive/GLAD_RESULTS/identifier_performance/'
        savepath=savepath+dataset_name+model_name.split('/')[1]
        adversarial_file = path + 'Identify' + dataset_name + '_' + model_name.split('/')[1] + '_dataset_final_try.csv'
        df = pd.read_csv(adversarial_file)
        df = df.drop_duplicates()
        df = df.fillna(0)
        # Define features and target variable
        features = ['expScoreAdv', 'left', 'right', 'replacescore', 'FreqImp', 'Errors', 'Freq', 'LabelChanged']
        X = df[features]
        y = df['Label']

        # Split the dataset into training and testing sets
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

        # Setting up the pipeline with SMOTE and XGBoost
        pipeline = imPipeline([
            ('smote', SMOTE(random_state=42)),
            ('classifier', xgb.XGBClassifier(random_state=42, use_label_encoder=False))
        ])

        # Define hyperparameters grid for RandomizedSearchCV
        param_grid = {
            'classifier__n_estimators': [100, 200, 500],
            'classifier__max_depth': [3, 5, 7, 10],
            'classifier__learning_rate': [0.01, 0.05, 0.1],
            'classifier__colsample_bytree': [0.6, 0.8, 1.0],
            'classifier__subsample': [0.6, 0.8, 1.0]
        }

        # Setup for RandomizedSearchCV
        cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
        random_search = RandomizedSearchCV(
            estimator=pipeline,
            param_distributions=param_grid,
            n_iter=10,
            scoring='balanced_accuracy',
            cv=cv,
            random_state=42
        )

        # Fit RandomizedSearchCV to the data
        random_search.fit(X_train, y_train)

        # Evaluating the best model from RandomizedSearchCV
        model = random_search.best_estimator_
        results = []
        overall_results = evaluate_model(model, X_test, y_test)
        overall_results['Type'] = 'Overall'
        results.append(overall_results)

        # Evaluate model for each attack type
        X_test_with_attack = X_test.copy()
        X_test_with_attack['Attack'] = df.loc[X_test.index, 'Attack']

        for attack in X_test_with_attack['Attack'].unique():
          if(attack!='textfooler'):
              idx = X_test_with_attack[X_test_with_attack['Attack'] == attack].index
              X_attack = X_test_with_attack.loc[idx].drop('Attack', axis=1)
              y_attack = y_test.loc[idx]

              attack_results = evaluate_model(model, X_attack, y_attack)
              attack_results['Type'] = attack
              results.append(attack_results)

        # Convert the list of dictionaries to a DataFrame
        results_df = pd.DataFrame(results)
        results_df.to_csv(savepath + 'Test_Seen_Attack_Types.csv', index=False)

        print(results_df)

        with open(savepath + 'XGBoost_identifier.pkl', 'wb') as file:
                pickle.dump(model, file)


In [ ]:
# Load the dataset
datasets=['sst2']

savepath='/content/gdrive/My Drive/GLAD_RESULTS/identifier_performance/'


for dataset_name in datasets:
  models=[]
  if(dataset_name in ['imdb','ag-news']):
    models.append('textattack/bert-base-uncased-'+dataset_name)

    models.append('textattack/roberta-base-'+dataset_name)

  elif(dataset_name =='sst2'):
    models.append('textattack/bert-base-uncased-SST-2')
    models.append('textattack/roberta-base-SST-2')

  else:
    models.append('textattack/bert-base-uncased-yelp-polarity')
    models.append('VictorSanh/roberta-base-finetuned-yelp-polarity')


  for model_name in models:
        path = '/content/gdrive/My Drive/GLAD_RESULTS/' + dataset_name + '/'
        savepath='/content/gdrive/My Drive/GLAD_RESULTS/identifier_performance/'
        savepath=savepath+dataset_name+model_name.split('/')[1]
        adversarial_file = path + 'Identify' + dataset_name + '_' + model_name.split('/')[1] + '_dataset_final_test.csv'
        # Load the test dataset
        print("MODEL ",model_name, " Dataset ", dataset_name)
        with open(savepath + 'XGBoost_identifier.pkl', 'rb') as file:
                identifier=pickle.load(file)
        df_test = pd.read_csv(adversarial_file)
        df_test = df_test.drop_duplicates()
        df_test = df_test.fillna(0)
        # Define the features and labels
        features = ['expScoreAdv', 'left', 'right', 'replacescore', 'FreqImp', 'Errors', 'Freq', 'LabelChanged']
        print("Features used: ", features)
        X_test = df_test[features]
        y_test = df_test['Label']
        print(adversarial_file)
        # Load the trained model
        # Make sure the model is loaded; assuming it's pickled or loaded elsewhere in your script
        # with open('path_to_model.pkl', 'rb') as file:
        #     model = pickle.load(file)

        # Evaluate the model overall
        overall_results= evaluate_model(identifier, X_test, y_test)
        overall_results['Type'] = 'Overall'

        # Initialize a list to store the results
        results = [overall_results]

        # Iterating over each unique attack type
        for attack in df_test['Attack'].unique():
            # Filter X_test for the current attack type, assuming 'Attack' is a column in df_test
            idx = df_test[df_test['Attack'] == attack].index
            X_attack = X_test.loc[idx]
            y_attack = y_test.loc[idx]

            # Evaluate the model
            attack_results= evaluate_model(identifier, X_attack, y_attack)

            # Create a report dictionary for the current attack type
            attack_results['Type'] = attack
            print(attack)
            results.append(attack_results)

        # Convert the list of dictionaries to a DataFrame
        results_df = pd.DataFrame(results)
        print(results_df)
        # Save the DataFrame to a CSV file
        results_df.to_csv(savepath + 'Test_UnSeen_Attack_Types_.csv', index=False)





MODEL  textattack/bert-base-uncased-SST-2  Dataset  sst2
Features used:  ['expScoreAdv', 'left', 'right', 'replacescore', 'FreqImp', 'Errors', 'Freq', 'LabelChanged']
/content/gdrive/My Drive/GLAD_RESULTS/sst2/Identifysst2_bert-base-uncased-SST-2_dataset_final_test.csv
Unique labels in y_test: [0 1]
Unique labels in predictions: [0 1]
Unique labels in y_test: [0 1]
Unique labels in predictions: [0 1]
textbugger
Unique labels in y_test: [0 1]
Unique labels in predictions: [0 1]
textfooler
Unique labels in y_test: [0 1]
Unique labels in predictions: [0 1]
tf-adj
Unique labels in y_test: [0 1]
Unique labels in predictions: [0 1]
a2t
        FPR       FNR       AUC  F1 Score    Recall  Precision  \
0  0.216535  0.369800  0.766266  0.678233  0.706832   0.667133   
1  0.184466  0.181818  0.885912  0.766622  0.816858   0.746795   
2  0.202397  0.291339  0.811663  0.727813  0.753132   0.716107   
3  0.268170  0.630303  0.585145  0.531638  0.550763   0.535328   
4  0.153153  0.413793  0.786683 

#Comparing with Baselines

In [ ]:
# Load the dataset
datasets=['imdb','sst2','ag-news','yelp_polarity']


for dataset_name in datasets:
  models=[]
  if(dataset_name in ['imdb','ag-news']):

    models.append('textattack/bert-base-uncased-'+dataset_name)
    models.append('textattack/roberta-base-'+dataset_name)
  elif(dataset_name =='sst2'):
    models.append('textattack/bert-base-uncased-SST-2')
    models.append('textattack/roberta-base-SST-2')

  else:
    models.append('textattack/bert-base-uncased-yelp-polarity')
    models.append('VictorSanh/roberta-base-finetuned-yelp-polarity')

  print(dataset_name)
  print(models)

  for model_name in models:
    features = ['expScoreAdv', 'replacescore', 'Freq']

    for f in features:
        path = '/content/gdrive/My Drive/GLAD_RESULTS/' + dataset_name + '/'
        savepath='/content/gdrive/My Drive/GLAD_RESULTS/identifier_performance/'
        savepath=savepath+dataset_name+model_name.split('/')[1]+f+"_considered_"
        adversarial_file = path + 'Identify' + dataset_name + '_' + model_name.split('/')[1] + '_dataset_final_try.csv'
        df = pd.read_csv(adversarial_file)
        df = df.drop_duplicates()
        df = df.fillna(0)
        # Define features and target variable
        print("Feature")
        X = df
        y = df['Label']

        # Split the dataset into training and testing sets
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

        # Setting up the pipeline with SMOTE and XGBoost
        pipeline = imPipeline([
            ('smote', SMOTE(random_state=42)),
            ('classifier', xgb.XGBClassifier(random_state=42, use_label_encoder=False))
        ])

        # Define hyperparameters grid for RandomizedSearchCV
        param_grid = {
            'classifier__n_estimators': [100, 200, 500],
            'classifier__max_depth': [3, 5, 7, 10],
            'classifier__learning_rate': [0.01, 0.05, 0.1],
            'classifier__colsample_bytree': [0.6, 0.8, 1.0],
            'classifier__subsample': [0.6, 0.8, 1.0]
        }

        # Setup for RandomizedSearchCV
        cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
        random_search = RandomizedSearchCV(
            estimator=pipeline,
            param_distributions=param_grid,
            n_iter=10,
            scoring='balanced_accuracy',
            cv=cv,
            random_state=42
        )
        X_train=X_train[f]
        X_attack=X_test
        # Fit RandomizedSearchCV to the data
        random_search.fit(np.array(X_train).reshape(-1,1), y_train)

        # Evaluating the best model from RandomizedSearchCV
        model = random_search.best_estimator_
        results = []
        overall_results = evaluate_model(model, np.array(X_test[f]).reshape(-1,1), y_test)
        overall_results['Type'] = 'Overall'
        results.append(overall_results)
        print(X_attack.columns)
        # Evaluate model for each attack type
        X_test_with_attack = X_attack.copy()[[f,'Attack']]
        X_test_with_attack['Attack'] = df.loc[X_attack.index, 'Attack']

        for attack in X_test_with_attack['Attack'].unique():
          if(attack!='textfooler'):
              idx = X_test_with_attack[X_test_with_attack['Attack'] == attack].index
              X_attack = X_test_with_attack.loc[idx].drop('Attack', axis=1)
              y_attack = y_test.loc[idx]

              attack_results = evaluate_model(model, np.array(X_attack).reshape(-1,1), y_attack)
              attack_results['Type'] = attack
              results.append(attack_results)

        # Convert the list of dictionaries to a DataFrame
        results_df = pd.DataFrame(results)
        results_df.to_csv(savepath + 'Test_Seen_Attack_Types.csv', index=False)

        print(results_df)

        with open(savepath + 'XGBoost_identifier.pkl', 'wb') as file:
                pickle.dump(model, file)


imdb
['textattack/bert-base-uncased-imdb', 'textattack/roberta-base-imdb']
Feature
Unique labels in y_test: [0 1]
Unique labels in predictions: [0 1]
Index(['words', 'expScoreAdv', 'left', 'right', 'gradient', 'replacescore',
       'FreqImp', 'Errors', 'Freq', 'LabelChanged', 'Label', 'Attack',
       'Example'],
      dtype='object')
Unique labels in y_test: [0 1]
Unique labels in predictions: [0 1]
Unique labels in y_test: [0 1]
Unique labels in predictions: [0 1]
Unique labels in y_test: [0 1]
Unique labels in predictions: [0 1]
        FPR       FNR       AUC  F1 Score    Recall  Precision  \
0  0.240841  0.504670  0.670831  0.558512  0.627245   0.561966   
1  0.258398  0.604034  0.591723  0.506818  0.568784   0.526923   
2  0.222502  0.179104  0.871408  0.617796  0.799197   0.609110   
3  0.227423  0.518519  0.675497  0.582929  0.627029   0.578600   

   Balanced Accuracy  Youden's Index         Type  
0           0.627245        0.254490      Overall  
1           0.568784      

<ipython-input-22-be22b07c0906>:30: DtypeWarning: Columns (23,24) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(adversarial_file)


Feature
Unique labels in y_test: [0 1]
Unique labels in predictions: [0 1]
Index(['words', 'expScoreAdv', 'left', 'right', 'gradient', 'replacescore',
       'FreqImp', 'Errors', 'Freq', 'LabelChanged', 'Label', 'Attack',
       'Example', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16',
       'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20',
       'Unnamed: 21', 'Unnamed: 22', 'Unnamed: 23', 'Unnamed: 24'],
      dtype='object')
Unique labels in y_test: [0 1]
Unique labels in predictions: [0 1]
Unique labels in y_test: [0 1]
Unique labels in predictions: [0 1]
Unique labels in y_test: [0 1]
Unique labels in predictions: [0 1]
        FPR       FNR       AUC  F1 Score    Recall  Precision  \
0  0.283854  0.370775  0.728799  0.549327  0.672686   0.566122   
1  0.266300  0.471810  0.677975  0.523330  0.630945   0.544227   
2  0.269912  0.350649  0.750798  0.582424  0.689720   0.587541   
3  0.368421  0.192308  0.786708  0.512048  0.719636   0.564288   

   Balanced

<ipython-input-22-be22b07c0906>:30: DtypeWarning: Columns (23,24) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(adversarial_file)


Feature
Unique labels in y_test: [0 1]
Unique labels in predictions: [0 1]
Index(['words', 'expScoreAdv', 'left', 'right', 'gradient', 'replacescore',
       'FreqImp', 'Errors', 'Freq', 'LabelChanged', 'Label', 'Attack',
       'Example', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16',
       'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20',
       'Unnamed: 21', 'Unnamed: 22', 'Unnamed: 23', 'Unnamed: 24'],
      dtype='object')
Unique labels in y_test: [0 1]
Unique labels in predictions: [0 1]
Unique labels in y_test: [0 1]
Unique labels in predictions: [0 1]
Unique labels in y_test: [0 1]
Unique labels in predictions: [0 1]
        FPR       FNR       AUC  F1 Score    Recall  Precision  \
0  0.306721  0.400596  0.690189  0.528427  0.646341   0.554433   
1  0.307858  0.427300  0.682800  0.506988  0.632421   0.541446   
2  0.314858  0.322820  0.732635  0.558822  0.681161   0.578370   
3  0.281547  0.653846  0.537793  0.478065  0.532303   0.511119   

   Balanced

<ipython-input-22-be22b07c0906>:30: DtypeWarning: Columns (23,24) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(adversarial_file)


Feature
Unique labels in y_test: [0 1]
Unique labels in predictions: [0 1]
Index(['words', 'expScoreAdv', 'left', 'right', 'gradient', 'replacescore',
       'FreqImp', 'Errors', 'Freq', 'LabelChanged', 'Label', 'Attack',
       'Example', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16',
       'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20',
       'Unnamed: 21', 'Unnamed: 22', 'Unnamed: 23', 'Unnamed: 24'],
      dtype='object')
Unique labels in y_test: [0 1]
Unique labels in predictions: [0 1]
Unique labels in y_test: [0 1]
Unique labels in predictions: [0 1]
Unique labels in y_test: [0 1]
Unique labels in predictions: [0 1]
        FPR       FNR       AUC  F1 Score    Recall  Precision  \
0  0.203022  0.507952  0.700361  0.571028  0.644513   0.567593   
1  0.196800  0.759644  0.552697  0.496040  0.521778   0.509382   
2  0.205403  0.452690  0.738824  0.601094  0.670953   0.591949   
3  0.213063  0.084615  0.933057  0.642090  0.851161   0.626376   

   Balanced

In [ ]:
# Load the dataset
datasets=['sst2']

savepath='/content/gdrive/My Drive/GLAD_RESULTS/identifier_performance/'


for dataset_name in datasets:
  models=[]
  if(dataset_name in ['imdb','ag-news']):
    models.append('textattack/bert-base-uncased-'+dataset_name)

    models.append('textattack/roberta-base-'+dataset_name)

  elif(dataset_name =='sst2'):
    models.append('textattack/bert-base-uncased-SST-2')
    models.append('textattack/roberta-base-SST-2')

  else:
    models.append('textattack/bert-base-uncased-yelp-polarity')
    models.append('VictorSanh/roberta-base-finetuned-yelp-polarity')


  for model_name in models:
    features = ['expScoreAdv', 'replacescore', 'Freq']

    for f in features:
        path = '/content/gdrive/My Drive/GLAD_RESULTS/' + dataset_name + '/'
        savepath='/content/gdrive/My Drive/GLAD_RESULTS/identifier_performance/'
        savepath=savepath+dataset_name+model_name.split('/')[1]+f+"_considered_"
        adversarial_file = path + 'Identify' + dataset_name + '_' + model_name.split('/')[1] + '_dataset_final_test.csv'
        # Load the test dataset
        print("MODEL ",model_name, " Dataset ", dataset_name)
        with open(savepath + 'XGBoost_identifier.pkl', 'rb') as file:
                identifier=pickle.load(file)
        df_test = pd.read_csv(adversarial_file)
        df_test = df_test.drop_duplicates()
        df_test = df_test.fillna(0)
        # Define the features and labels
        features = ['expScoreAdv', 'left', 'right', 'replacescore', 'FreqImp', 'Errors', 'Freq', 'LabelChanged']
        print("Features used: ", f)
        X_test = df_test[f]
        y_test = df_test['Label']

        # Load the trained model
        # Make sure the model is loaded; assuming it's pickled or loaded elsewhere in your script
        # with open('path_to_model.pkl', 'rb') as file:
        #     model = pickle.load(file)

        # Evaluate the model overall
        overall_results= evaluate_model(identifier, np.array(X_test).reshape(-1,1), y_test)
        overall_results['Type'] = 'Overall'

        # Initialize a list to store the results
        results = [overall_results]

        # Iterating over each unique attack type
        for attack in df_test['Attack'].unique():
            # Filter X_test for the current attack type, assuming 'Attack' is a column in df_test
            idx = df_test[df_test['Attack'] == attack].index
            X_attack = X_test.loc[idx]
            y_attack = y_test.loc[idx]

            # Evaluate the model
            attack_results= evaluate_model(identifier, X_attack, y_attack)

            # Create a report dictionary for the current attack type
            attack_results['Type'] = attack
            print(attack)
            results.append(attack_results)

        # Convert the list of dictionaries to a DataFrame
        results_df = pd.DataFrame(results)
        print(results_df)
        # Save the DataFrame to a CSV file
        results_df.to_csv(savepath + 'Test_UnSeen_Attack_Types_.csv', index=False)





MODEL  textattack/bert-base-uncased-SST-2  Dataset  sst2


In [ ]:
# Load the dataset
datasets=['ag-news','imdb','yelp_polarity','sst2']

savepath='/content/gdrive/My Drive/GLAD_RESULTS/identifier_performance/'


for dataset_name in datasets:
  models=[]
  if(dataset_name in ['imdb','ag-news']):
    models.append('textattack/bert-base-uncased-'+dataset_name)

    models.append('textattack/roberta-base-'+dataset_name)

  elif(dataset_name =='sst2'):
    models.append('textattack/bert-base-uncased-SST-2')
    models.append('textattack/roberta-base-SST-2')

  else:
    models.append('textattack/bert-base-uncased-yelp-polarity')
    models.append('VictorSanh/roberta-base-finetuned-yelp-polarity')


  for model_name in models:
        path = '/content/gdrive/My Drive/GLAD_RESULTS/' + dataset_name + '/'
        savepath='/content/gdrive/My Drive/GLAD_RESULTS/identifier_performance/'
        savepath=savepath+dataset_name+model_name.split('/')[1]
        adversarial_file = path + 'Identify' + dataset_name + '_' + model_name.split('/')[1] + '_dataset_final_test.csv'
        # Load the test dataset
        print("MODEL ",model_name, " Dataset ", dataset_name)
        with open(savepath + 'XGBoost_identifier.pkl', 'rb') as file:
                identifier=pickle.load(file)
        df_test = pd.read_csv(adversarial_file)
        df_test = df_test.drop_duplicates()
        df_test = df_test.fillna(0)
        # Define the features and labels
        features = ['expScoreAdv', 'left', 'right', 'replacescore', 'FreqImp', 'Errors', 'Freq', 'LabelChanged']
        print("Features used: ", features)
        X_test = df_test[features]
        y_test = df_test['Label']
        print(adversarial_file)
        # Load the trained model
        # Make sure the model is loaded; assuming it's pickled or loaded elsewhere in your script
        # with open('path_to_model.pkl', 'rb') as file:
        #     model = pickle.load(file)

        # Evaluate the model overall
        overall_results= evaluate_model(identifier, X_test, y_test)
        overall_results['Type'] = 'Overall'

        # Initialize a list to store the results
        results = [overall_results]

        # Iterating over each unique attack type
        for attack in df_test['Attack'].unique():
            # Filter X_test for the current attack type, assuming 'Attack' is a column in df_test
            idx = df_test[df_test['Attack'] == attack].index
            X_attack = X_test.loc[idx]
            y_attack = y_test.loc[idx]

            # Evaluate the model
            attack_results= evaluate_model(identifier, X_attack, y_attack)

            # Create a report dictionary for the current attack type
            attack_results['Type'] = attack
            print(attack)
            results.append(attack_results)

        # Convert the list of dictionaries to a DataFrame
        results_df = pd.DataFrame(results)
        print(results_df)
        # Save the DataFrame to a CSV file
        results_df.to_csv(savepath + 'Test_UnSeen_Attack_Types_.csv', index=False)



